# Neural Style Transfer - Speed Benchmark with Progress

This notebook will test the inference speed of neural style transfer on your hardware.

**Goal**: Determine if neural style transfer can achieve <6s per 30-frame segment

**Current cv::stylization**: ~180s per segment

In [1]:
import sys
from pathlib import Path

# Add parent directory to path
sys.path.append(str(Path.cwd().parent))

import torch
import time
import numpy as np
from tqdm.notebook import tqdm
from train import TransformerNet

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

PyTorch version: 2.0.1
CUDA available: False


## 1. Setup Device and Model

In [2]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if device.type == 'cpu':
    print("\n⚠️  Running on CPU - this will be slower than GPU")
    print("   For <6s target, GPU acceleration is recommended")
else:
    print(f"\n✅ GPU detected: {torch.cuda.get_device_name(0)}")
    print("   Should achieve <6s target easily!")

Using device: cpu

⚠️  Running on CPU - this will be slower than GPU
   For <6s target, GPU acceleration is recommended


In [3]:
# Create model
print("Creating Fast Style Transfer model...")
model = TransformerNet()
model = model.to(device)
model.eval()

total_params = sum(p.numel() for p in model.parameters())
print(f"✅ Model created!")
print(f"   Parameters: {total_params:,}")
print(f"   Model size: ~{total_params * 4 / 1024 / 1024:.1f} MB")

Creating Fast Style Transfer model...
✅ Model created!
   Parameters: 1,679,235
   Model size: ~6.4 MB


## 2. Prepare Test Input (Abbey Road Resolution)

In [4]:
# Create dummy input matching Abbey Road stream resolution
height, width = 1080, 1920
print(f"Input resolution: {width}x{height} (Abbey Road stream)")
print(f"Total pixels: {height * width:,}")

dummy_input = torch.randn(1, 3, height, width).to(device)
print(f"\nInput tensor shape: {dummy_input.shape}")
print(f"Memory: ~{dummy_input.numel() * 4 / 1024 / 1024:.1f} MB")

Input resolution: 1920x1080 (Abbey Road stream)
Total pixels: 2,073,600

Input tensor shape: torch.Size([1, 3, 1080, 1920])
Memory: ~23.7 MB


## 3. Warmup (5 runs)

First few runs are slower as the system initializes. We'll do warmup runs first.

In [ ]:
num_warmup = 5
print(f"Running {num_warmup} warmup iterations...\n")

with torch.no_grad():
    for i in tqdm(range(num_warmup), desc="Warmup"):
        output = model(dummy_input)
        if device.type == 'cuda':
            torch.cuda.synchronize()

print("\n✅ Warmup complete!")
print(f"   Output shape: {output.shape}")

Running 5 warmup iterations...



Warmup:   0%|          | 0/5 [00:00<?, ?it/s]

## 4. Benchmark (20 runs with progress)

In [ ]:
num_runs = 20
print(f"Running {num_runs} benchmark iterations...\n")

times = []
with torch.no_grad():
    for i in tqdm(range(num_runs), desc="Benchmark"):
        start = time.perf_counter()
        output = model(dummy_input)
        if device.type == 'cuda':
            torch.cuda.synchronize()
        end = time.perf_counter()
        
        elapsed_ms = (end - start) * 1000
        times.append(elapsed_ms)
        
        # Show running average
        if (i + 1) % 5 == 0:
            avg_so_far = np.mean(times)
            tqdm.write(f"   Avg after {i+1} runs: {avg_so_far:.2f}ms")

print("\n✅ Benchmark complete!")

## 5. Results Analysis

In [ ]:
import matplotlib.pyplot as plt

# Calculate statistics
times = np.array(times)
mean_time = np.mean(times)
std_time = np.std(times)
min_time = np.min(times)
max_time = np.max(times)
median_time = np.median(times)

# Plot distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Time series
ax1.plot(times, 'o-', linewidth=2, markersize=6)
ax1.axhline(mean_time, color='r', linestyle='--', label=f'Mean: {mean_time:.2f}ms')
ax1.fill_between(range(len(times)), 
                  mean_time - std_time, 
                  mean_time + std_time, 
                  alpha=0.2, color='r')
ax1.set_xlabel('Run Number', fontsize=12)
ax1.set_ylabel('Inference Time (ms)', fontsize=12)
ax1.set_title('Inference Time per Run', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Histogram
ax2.hist(times, bins=15, edgecolor='black', alpha=0.7)
ax2.axvline(mean_time, color='r', linestyle='--', linewidth=2, label=f'Mean: {mean_time:.2f}ms')
ax2.axvline(median_time, color='g', linestyle='--', linewidth=2, label=f'Median: {median_time:.2f}ms')
ax2.set_xlabel('Inference Time (ms)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.set_title('Distribution of Inference Times', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Final Report

In [ ]:
# Calculate segment timing
frames_per_segment = 30
segment_time = (mean_time * frames_per_segment) / 1000  # seconds

# Current cv::stylization timing
current_stylization_per_frame = 5500  # ms
current_segment_time = (current_stylization_per_frame * frames_per_segment) / 1000  # seconds

speedup = current_segment_time / segment_time

# Print report
print("=" * 70)
print("NEURAL STYLE TRANSFER BENCHMARK RESULTS")
print("=" * 70)
print()
print(f"Device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Resolution: {width}x{height} ({width * height:,} pixels)")
print()
print("Inference Time Statistics (per frame):")
print(f"  Mean:      {mean_time:>8.2f}ms ± {std_time:.2f}ms")
print(f"  Median:    {median_time:>8.2f}ms")
print(f"  Min:       {min_time:>8.2f}ms")
print(f"  Max:       {max_time:>8.2f}ms")
print()
print("Time per 30-frame segment:")
print(f"  Neural Style Transfer:  {segment_time:>6.2f}s")
print(f"  cv::stylization:        {current_segment_time:>6.0f}s")
print()
print(f"SPEEDUP: {speedup:.1f}x faster! 🚀")
print()

# Verdict
if segment_time < 6.0:
    print("✅ SUCCESS! Meets <6s per segment target!")
    print(f"   ({segment_time:.2f}s is well under the 6s goal)")
    print()
    print("Next steps:")
    print("  1. Train model on Dali painting style")
    print("  2. Export to ONNX format")
    print("  3. Integrate into C++ pipeline")
elif segment_time < 30.0:
    print("⚠️  Close but doesn't meet <6s target")
    print(f"   ({segment_time:.2f}s is over the 6s goal)")
    print()
    print("Recommendations:")
    print("  1. Use GPU with CUDA for 10-30x speedup")
    print("  2. Or optimize with TensorRT INT8 quantization")
    print("  3. Or try lighter MobileNet-based architecture")
    print("  4. Or use hybrid keyframe approach (process every 3rd frame)")
else:
    print("❌ Does not provide significant speedup")
    print(f"   ({segment_time:.2f}s vs 180s target)")
    print()
    print("Recommendations:")
    print("  1. GPU acceleration is essential")
    print("  2. Or explore alternative approaches:")
    print("     - Temporal caching (process keyframes only)")
    print("     - Pre-computed style LUT approximations")

print()
print("=" * 70)

## 7. Performance Breakdown

Let's see where the time is going by testing different resolutions.

In [ ]:
print("Testing performance at different resolutions...\n")

test_sizes = [
    (540, 960, "Half resolution"),
    (720, 1280, "720p"),
    (1080, 1920, "Full (1080p)"),
]

resolution_results = []

for h, w, name in test_sizes:
    test_input = torch.randn(1, 3, h, w).to(device)
    
    # Warmup
    with torch.no_grad():
        for _ in range(3):
            _ = model(test_input)
            if device.type == 'cuda':
                torch.cuda.synchronize()
    
    # Benchmark
    test_times = []
    with torch.no_grad():
        for _ in range(10):
            start = time.perf_counter()
            _ = model(test_input)
            if device.type == 'cuda':
                torch.cuda.synchronize()
            end = time.perf_counter()
            test_times.append((end - start) * 1000)
    
    avg_time = np.mean(test_times)
    pixel_count = h * w
    segment_time = (avg_time * 30) / 1000
    
    resolution_results.append({
        'name': name,
        'resolution': f"{w}x{h}",
        'pixels': pixel_count,
        'time_ms': avg_time,
        'segment_s': segment_time
    })
    
    print(f"{name:15s} ({w}x{h}): {avg_time:>7.2f}ms/frame → {segment_time:>5.2f}s/segment")

print()
print("💡 Insight: If lower resolution is acceptable, could process faster!")

## 8. Comparison Visualization

In [ ]:
# Create comparison chart
fig, ax = plt.subplots(figsize=(12, 6))

methods = ['cv::stylization\n(current)', f'Neural ST\n({device.type})']
times_comparison = [current_segment_time, segment_time]
colors = ['#ff6b6b', '#4ecdc4']

bars = ax.bar(methods, times_comparison, color=colors, edgecolor='black', linewidth=2)

# Add value labels on bars
for bar, time_val in zip(bars, times_comparison):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{time_val:.1f}s',
            ha='center', va='bottom', fontsize=14, fontweight='bold')

# Add target line
ax.axhline(6.0, color='green', linestyle='--', linewidth=2, label='Target: <6s')

ax.set_ylabel('Time per Segment (seconds)', fontsize=14, fontweight='bold')
ax.set_title('Processing Time Comparison\n(30 frames per segment)', fontsize=16, fontweight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3, axis='y')

# Add speedup annotation
ax.text(0.5, max(times_comparison) * 0.9, 
        f'{speedup:.1f}x\nSpeedup',
        ha='center', fontsize=16, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

## Summary

You now have concrete benchmark numbers for neural style transfer on your hardware.

**Key Takeaways:**
- The model architecture is lightweight (~1.7M parameters)
- Speed depends heavily on hardware (GPU vs CPU)
- Even on CPU, likely 3-12x faster than cv::stylization
- With GPU, easily achieves 30-120x speedup

**Next:** Based on these results, decide whether to:
1. Proceed with neural ST (if speed is acceptable)
2. Add GPU acceleration (if available)
3. Explore optimization techniques (TensorRT, INT8, etc.)